In [1]:
import pandas as pd
import pprint
import os
import numpy as np
import time
import re
import torch
import torch.nn as nn
from torch.autograd import Variable

In [3]:
df1 = pd.read_csv('lotto.csv'
                   , engine='python'                   
                   , names = ['1','2','3','4','5','6','7']
                   #, encoding='utf-8'                   
                   #, na_filter =True                   
                  )
df1=df1.drop(['7'],axis=1)
df1



,1,2,3,4,5,6
0,1,3,12,14,16,43
1,8,11,19,21,36,45
2,5,10,13,21,39,43
3,6,11,15,17,23,40
4,15,21,31,32,41,43
5,6,10,18,25,34,35
6,14,20,23,31,37,38
7,3,12,13,18,31,32
8,1,10,13,26,32,36
9,5,9,14,26,30,43


In [4]:
lotto_data = []
for idx in range(len(df1)-10):
    lotto_data.append(df1.loc[idx].tolist())

In [5]:
len(lotto_data)

802

In [6]:
len(lotto_data),lotto_data

(802,
 [[1, 3, 12, 14, 16, 43],
  [8, 11, 19, 21, 36, 45],
  [5, 10, 13, 21, 39, 43],
  [6, 11, 15, 17, 23, 40],
  [15, 21, 31, 32, 41, 43],
  [6, 10, 18, 25, 34, 35],
  [14, 20, 23, 31, 37, 38],
  [3, 12, 13, 18, 31, 32],
  [1, 10, 13, 26, 32, 36],
  [5, 9, 14, 26, 30, 43],
  [10, 11, 12, 18, 24, 42],
  [17, 25, 28, 37, 43, 44],
  [1, 4, 10, 12, 28, 45],
  [12, 17, 23, 34, 42, 45],
  [2, 10, 14, 22, 32, 36],
  [5, 22, 31, 32, 39, 45],
  [1, 21, 26, 36, 40, 41],
  [3, 10, 13, 26, 34, 38],
  [6, 7, 18, 19, 30, 38],
  [10, 15, 21, 35, 38, 43],
  [2, 7, 19, 25, 29, 36],
  [2, 10, 12, 31, 33, 42],
  [3, 8, 19, 27, 30, 41],
  [2, 6, 7, 12, 19, 45],
  [2, 10, 11, 19, 35, 39],
  [5, 6, 13, 16, 27, 28],
  [12, 15, 16, 20, 24, 30],
  [4, 6, 15, 25, 26, 33],
  [3, 10, 23, 24, 31, 39],
  [14, 15, 16, 17, 38, 45],
  [6, 18, 31, 34, 38, 45],
  [11, 16, 18, 19, 24, 39],
  [15, 17, 19, 21, 27, 45],
  [6, 12, 19, 24, 34, 41],
  [6, 21, 35, 36, 37, 41],
  [6, 12, 17, 21, 34, 37],
  [8, 9, 18, 21, 28, 4

In [7]:
char_dic = {w: i for i, w in enumerate(range(1,46))}
char_dic

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44}

In [20]:
dataX = []
dataY = []
for i in range(len(lotto_data)-1):
    x_str = lotto_data[i]
    y_str = lotto_data[i+1]
    #print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index
    
    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

x_data = torch.Tensor(dataX)
y_data = torch.LongTensor(dataY)


In [10]:
x_data[0]

tensor([  0.,   2.,  11.,  13.,  15.,  42.])

In [21]:
input_size = len(char_dic)        # RNN input size (one hot size)
hidden_size = len(char_dic)     # RNN output size
num_classes = len(char_dic)   # final output size (RNN or softmax, etc.)
# one hot encoding
def one_hot(x, num_classes):
    idx = x.long()
    idx = idx.view(-1, 1)

    x_one_hot = torch.zeros(x.size()[0] * x.size()[1], num_classes)
    x_one_hot.scatter_(1, idx, 1)
    x_one_hot = x_one_hot.view(x.size()[0], x.size()[1], num_classes)
    return x_one_hot


x_one_hot = one_hot(x_data, num_classes)

inputs = Variable(x_one_hot)
labels = Variable(y_data)


In [22]:
inputs[0],labels[0]

(tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [26]:
# hyperparameters
learning_rate = 0.005
num_epochs = 10000
sequence_length = 6            # any arbitrary number
num_layers = 3                      # number of layers in RNN

class LSTM(nn.Module):
 
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        
        # Set parameters for RNN block
        # Note: batch_first=False by default.
        # When true, inputs are (batch_size, sequence_length, input_dimension)
        # instead of (sequence_length, batch_size, input_dimension)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers,batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden and cell states
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        # Input: (batch, seq_len, input_size)
        out, _ = self.lstm(x, (h_0, c_0))
        
        # Note: the output tensor of LSTM in this case is a block with holes
        # > add .contiguous() to apply view()
        out = out.contiguous().view(-1, self.hidden_size)
        
        # Return outputs applied to fully connected layer
        out = self.fc(out)
        
        return out


In [27]:
# Instantiate RNN model
lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

# Set loss and optimizer function
criterion = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [28]:
# Train the model
for epoch in range(num_epochs):
    outputs = lstm(inputs)
    optimizer.zero_grad()
    
    # obtain the loss function
    # flatten target labels to match output
    loss = criterion(outputs, labels.view(-1))
    loss.backward()
    optimizer.step()
    
    # obtain the predicted indices of the next character
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    idx = idx.reshape(-1, sequence_length)  # (170,10)
    
    # display the prediction of the last sequence 
    if ((epoch + 1) % 100) ==1:
        print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))
        #result_str = [char_set[c] for c in idx[-2]]
        #print("Predicted string: ", ''.join(str(result_str)))

print("Learning finished!")


/home/oj-pjt-03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch: 1, loss: 3.810
epoch: 101, loss: 3.129
epoch: 201, loss: 3.086
epoch: 301, loss: 2.995
epoch: 401, loss: 2.832
epoch: 501, loss: 2.607
epoch: 601, loss: 2.343
epoch: 701, loss: 2.076
epoch: 801, loss: 1.807
epoch: 901, loss: 1.679
epoch: 1001, loss: 1.376
epoch: 1101, loss: 1.210
epoch: 1201, loss: 1.083
epoch: 1301, loss: 1.014
epoch: 1401, loss: 0.907
epoch: 1501, loss: 0.953
epoch: 1601, loss: 0.821
epoch: 1701, loss: 0.790
epoch: 1801, loss: 0.766
epoch: 1901, loss: 0.748
epoch: 2001, loss: 0.733
epoch: 2101, loss: 0.722
epoch: 2201, loss: 0.713
epoch: 2301, loss: 0.706
epoch: 2401, loss: 0.701
epoch: 2501, loss: 0.697
epoch: 2601, loss: 0.693
epoch: 2701, loss: 0.690
epoch: 2801, loss: 0.688
epoch: 2901, loss: 0.686
epoch: 3001, loss: 0.685
epoch: 3101, loss: 0.683
epoch: 3201, loss: 0.682
epoch: 3301, loss: 0.681
epoch: 3401, loss: 0.682
epoch: 3501, loss: 0.680
epoch: 3601, loss: 0.679
epoch: 3701, loss: 7.104
epoch: 3801, loss: 0.695
epoch: 3901, loss: 0.687
epoch: 4001,

In [29]:
testdf = []
for idx in range(len(df1)-11,len(df1)):
    testdf.append(df1.loc[idx].tolist())

In [30]:
testdf

[[1, 7, 36, 37, 41, 42],
 [9, 25, 30, 33, 41, 44],
 [2, 4, 16, 17, 36, 39],
 [8, 19, 25, 34, 37, 39],
 [2, 9, 16, 25, 26, 40],
 [14, 15, 26, 27, 40, 42],
 [16, 24, 29, 40, 41, 42],
 [14, 27, 30, 31, 40, 42],
 [11, 16, 19, 21, 27, 31],
 [9, 13, 21, 25, 32, 42],
 [10, 23, 29, 33, 37, 40]]

In [31]:
test = torch.Tensor(testdf)
test_one_hot = one_hot(test, num_classes)

dataT=Variable(test_one_hot)

In [32]:
outputs = lstm(dataT)
_, idx = outputs.max(1)


In [33]:
(idx)

tensor([  1,   7,  12,  30,  31,  37,   1,  14,  18,  14,  25,  29,
          0,   4,  36,  25,  25,  44,   0,   6,  24,  34,  34,  42,
          0,   6,  27,  27,  35,  35,   7,  10,  29,  25,  34,  37,
          0,   4,  14,  17,  34,  34,   7,   3,   7,  13,  26,  35,
          3,   9,  19,  19,  26,  40,   1,   8,  14,  18,  30,  44,
          3,   3,  17,  16,  28,  36])

In [34]:
set_test = np.array(idx)

In [36]:
set_test.reshape(11,6)+1

array([[ 2,  8, 13, 31, 32, 38],
       [ 2, 15, 19, 15, 26, 30],
       [ 1,  5, 37, 26, 26, 45],
       [ 1,  7, 25, 35, 35, 43],
       [ 1,  7, 28, 28, 36, 36],
       [ 8, 11, 30, 26, 35, 38],
       [ 1,  5, 15, 18, 35, 35],
       [ 8,  4,  8, 14, 27, 36],
       [ 4, 10, 20, 20, 27, 41],
       [ 2,  9, 15, 19, 31, 45],
       [ 4,  4, 18, 17, 29, 37]])

In [ ]:
[[1, 7, 36, 37, 41, 42, 14],
 [9, 25, 30, 33, 41, 44, 6],
 [2, 4, 16, 17, 36, 39, 14],
 [8, 19, 25, 34, 37, 39, 9],
 [2, 9, 16, 25, 26, 40, 42],
 [14, 15, 26, 27, 40, 42, 34],
 [16, 24, 29, 40, 41, 42, 3],
 [14, 27, 30, 31, 40, 42, 2],
 [11, 16, 19, 21, 27, 31, 30],
 [9, 13, 21, 25, 32, 42, 2],
 [10, 23, 29, 33, 37, 40, 16]]

In [ ]:
[[ 2,  8, 12, 19, 33, 33, 11],
 [ 2, 13, 10, 16, 26, 36, 30],
 [ 1,  6,  9, 11, 30, 38, 32],
 [ 1,  7, 18, 29, 35, 44, 20],
 [ 1,  7, 18, 27, 30, 39, 43],
 [ 6,  4, 18, 41, 43, 30, 13],
 [ 1,  4,  9, 12, 22, 28,  1],
 [ 6, 17, 18, 11, 35, 30, 29],
 [ 6, 10, 16, 27, 41, 45, 17],
 [ 2,  8, 15, 17, 27, 32, 13],
 [ 4,  4, 17, 20, 35, 39, 22]]